In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne 
from nice.markers import (KolmogorovComplexity, TimeLockedContrast, PowerSpectralDensityEstimator, PowerSpectralDensitySummary,
                          PowerSpectralDensity, SymbolicMutualInformation, PermutationEntropy, TimeLockedTopography, ContingentNegativeVariation)

import sys
sys.path.insert(0, '../')


# Compute markers function

In [3]:
def all_markers(epochs, tmin, tmax, target, epochs_erp = None):
        """
        Computes all ther markers for given epochs.
        epochs: the epochs from which to compute the markers
        tmin: min time for computing markers 
        tmax: max time to compute markers
        target: reduction target, epochs or topography
        epochs_erp: if a different preprocessed epochs are used for computing ERP
        
        Evoked markers have already defined times
        """
        if epochs_erp ==None:
            epochs_erp = epochs
        
        from scipy.stats import trim_mean
        
        def trim_mean80(a, axis=0):
            return trim_mean(a, proportiontocut=.1, axis=axis)
        
        def entropy(a, axis=0):  # noqa
            return -np.nansum(a * np.log(a), axis=axis) / np.log(a.shape[axis])

        # =============================================================================
        # SPECTRAL MARKERS
        # =============================================================================
          #PowerSpectralDensityL
        psds_params = dict(n_fft=4096, n_overlap=100, n_jobs='auto', nperseg=128)
        base_psd = PowerSpectralDensityEstimator(
            psd_method='welch', tmin=tmin, tmax=tmax, fmin=1., fmax=45.,
            psd_params=psds_params, comment='default')
        


        reduction_func = [{'axis': 'frequency', 'function': np.sum},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': np.mean}]
        
        ###alpha normalized###
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alpha')
        alpha.fit(epochs)
        dataalpha_n = alpha._reduce_to(reduction_func, target=target, picks=None)

        #alpha
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=False, comment='alpha')
        alpha.fit(epochs)
        dataalpha = alpha._reduce_to(reduction_func, target=target, picks=None)

        #delta normalized
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4.,normalize=True, comment='delta')
        delta.fit(epochs)
        datadelta_n = delta._reduce_to(reduction_func, target=target, picks=None)


        #delta
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4,normalize=False, comment='delta')
        delta.fit(epochs)
        datadelta = delta._reduce_to(reduction_func, target=target, picks=None)

        #theta normalized 
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,normalize=True, comment='theta')
        theta.fit(epochs)
        datatheta_n = theta._reduce_to(reduction_func, target=target, picks= None)


        #theta
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8,normalize=False, comment='theta')
        theta.fit(epochs)
        datatheta = theta._reduce_to(reduction_func, target=target, picks=None)

#       theta normalized Frontal
        frontal_chs= ['AF3', 'AFz', 'AF4', 'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2']
        frontal_chs = ['Fz']
        roi_frontal = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=frontal_chs))
        
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,normalize=True, comment='theta')
        theta.fit(epochs)
        datafrontaltheta_n = theta._reduce_to(reduction_func, target=target,picks = {'epochs': None,'channels': roi_frontal})


        #theta FRONTAL
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8,normalize=False, comment='theta')
        theta.fit(epochs)
        datafrontaltheta = theta._reduce_to(reduction_func, target=target,picks = {'epochs': None,'channels': roi_frontal})

        #gamma normalized
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45.,normalize=True, comment='gamma')
        gamma.fit(epochs)
        datagamma_n = gamma._reduce_to(reduction_func, target=target, picks=None)


        #gamma
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45,normalize=False, comment='theta')
        gamma.fit(epochs)
        datagamma = gamma._reduce_to(reduction_func, target=target, picks=None)

        #beta normalized
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30.,normalize=True, comment='beta')
        beta.fit(epochs)
        databetaa_n = beta._reduce_to(reduction_func, target=target, picks=None)


        #beta
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30,normalize=False, comment='beta')
        beta.fit(epochs)
        databeta = beta._reduce_to(reduction_func, target=target, picks=None)
        
        
        #Spectral Entropy
        se = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=45.,
                         normalize=True, comment='summary_se')
        se.fit(epochs)
        
        reduction_func = [{'axis': 'frequency', 'function': entropy},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': np.mean}]
        
        datase = se._reduce_to(reduction_func, target=target, picks=None)
        
        
        #### Spectral Summary ####
        
        reduction_func= [{'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': np.mean}]
        
        # msf
        msf = PowerSpectralDensitySummary(estimator=base_psd, fmin=1., fmax=45.,
                                percentile=.5, comment='summary_msf')
        msf.fit(epochs)
        datamsf = msf._reduce_to(reduction_func, target=target, picks=None)
        
        #sef90
        sef90 = PowerSpectralDensitySummary(estimator=base_psd, fmin=1., fmax=45.,
                                percentile=.9, comment='summary_sef90')
        sef90.fit(epochs)
        datasef90 = sef90._reduce_to(reduction_func, target=target, picks=None)
        
        #sef95
        sef95 = PowerSpectralDensitySummary(estimator=base_psd, fmin=1., fmax=45.,
                                percentile=.95, comment='summary_sef95')
        sef95.fit(epochs)
        datasef95 = sef95._reduce_to(reduction_func, target=target, picks=None)

        # =============================================================================
        # INFORMATION THEORY MARKERS
        # =============================================================================
        
        ### Kolgomorov complexity ###
        komplexity = KolmogorovComplexity(tmin=tmin, tmax=tmax, backend='openmp')
        komplexity.fit(epochs)
#         komplexityobject=komplexity.data_ ###Object to save, number of channels*number of epochs, it's ndarray
        reduction_func= [{'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': np.mean}]


        datakomplexity = komplexity._reduce_to(reduction_func, target=target, picks=None)

        
        
        
        ### Permuttion entropy ###
        p_e = PermutationEntropy(tmin=tmin, tmax=tmax, kernel=3, tau=1)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e1 = p_e._reduce_to(reduction_func, target=target, picks=None)
        
        ### Permuttion entropy ###
        p_e = PermutationEntropy(tmin=tmin, tmax=tmax, kernel=3, tau=2)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e2 = p_e._reduce_to(reduction_func, target=target, picks=None)
        
        ### Permuttion entropy ###
        p_e = PermutationEntropy(tmin=tmin, tmax=tmax, kernel=3, tau=4)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e4 = p_e._reduce_to(reduction_func, target=target, picks=None)
        
        ### Permuttion entropy theta###
        p_e = PermutationEntropy(tmin=tmin, tmax=tmax, kernel=3, tau=8)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e8 = p_e._reduce_to(reduction_func, target=target, picks=None)

        # =============================================================================
        # wSMI MARKERS
        # =============================================================================
        reduction_func= [{'axis': 'channels_y', 'function': np.median},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': np.mean}]
        
        ###wSMI ###
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=1, backend="python",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_
        datawSMI1 = wSMI._reduce_to(reduction_func, target=target, picks=None)
        
        ###wSMI ###
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=2, backend="python",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_
        datawSMI2 = wSMI._reduce_to(reduction_func, target=target, picks=None)
        
        ###wSMI ###
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=4, backend="python",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_
        datawSMI4 = wSMI._reduce_to(reduction_func, target=target, picks=None)
        
        ###wSMI ###
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=8, backend="python",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_
        datawSMI8 = wSMI._reduce_to(reduction_func, target=target, picks=None)
        
        
        # =============================================================================
        # EVOKED MARKERS
        # =============================================================================
        
        ###Contingent Negative Variation (CNV)###
        cnv = ContingentNegativeVariation(tmin=-0.004, tmax=0.596)
        
        reduction_func = [{'axis': 'epochs', 'function': np.mean},
             {'axis': 'channels', 'function': np.mean}]
        
        cnv.fit(epochs_erp)
        cnv_chs= ['AF3', 'AFz', 'AF4', 'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2']
        roi_cnv = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=cnv_chs))
        dataCNV = cnv._reduce_to(reduction_func, target=target, picks={
        'epochs': None,
        'channels': roi_cnv})
        
        ###P1###
        reduction_func = [{'axis': 'epochs', 'function': trim_mean80},
         {'axis': 'channels', 'function': np.mean},
         {'axis': 'times', 'function': np.mean}]
        p1 = TimeLockedTopography(tmin=0.068, tmax=0.116, comment='p1')
        p1.fit(epochs_erp)
        p1_chs= ['AF3', 'AFz', 'AF4', 'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2']
        roi_p1 = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=p1_chs))
        dataP1 = p1._reduce_to(reduction_func, target=target, picks={
        'epochs': None,
        'channels': roi_p1,
        'times':None})
        
        ###P3a###
        p3a = TimeLockedTopography(tmin=0.28, tmax=0.34, comment='p3a')
        reduction_func = [{'axis': 'epochs', 'function': trim_mean80},
         {'axis': 'channels', 'function': np.mean},
         {'axis': 'times', 'function': np.mean}]
        p3a.fit(epochs_erp)
        p3a_chs= ['AF3', 'AFz', 'AF4', 'F1', 'Fz', 'F2', 'FC1', 'FCz', 'FC2']
        roi_p3a = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=p3a_chs))
        dataP3a= p3a._reduce_to(reduction_func, target=target, picks={
        'epochs': None,
        'channels': roi_p3a,
        'times':None})
        
        ###P3b###
        p3b = TimeLockedTopography(tmin=0.4, tmax=0.6, comment='p3b')
        reduction_func = [{'axis': 'epochs', 'function': trim_mean80},
         {'axis': 'channels', 'function': np.mean},
         {'axis': 'times', 'function': np.mean}]
        p3b.fit(epochs_erp)
        p3b_chs= ['FC1', 'FCz', 'FC2', 'C1', 'Cz','C2', 'CP1', 'CPz', 'CP2']
        roi_p3b = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=p3b_chs))
        dataP3b= p3b._reduce_to(reduction_func, target=target, picks={
        'epochs': None,
        'channels': roi_p3b,
        'times':None})
        
        ###Dictionary with all the markers###
        return {'wSMI_1':datawSMI1,'wSMI_2':datawSMI2,'wSMI_4':datawSMI4,'wSMI_8':datawSMI8, 
                'p_e_1':datap_e1,'p_e_2':datap_e2,'p_e_4':datap_e4,'p_e_8':datap_e8, 
                'k':datakomplexity, 'se':datase,
                'msf': datamsf, 'sef90':datasef90, 'sef95':datasef95,
                'b':databeta,'b_n':databetaa_n, 'g':datagamma, 'g_n':datagamma_n,
                'ft': datafrontaltheta, 'ft_n':datafrontaltheta_n,
                't':datatheta,'t_n': datatheta_n , 'd':datadelta, 'd_n':datadelta_n, 
                'a_n':dataalpha_n, 'a':dataalpha, 
                'CNV':dataCNV, 'P1':dataP1, 'P3a': dataP3a, 'P3b': dataP3b}


In [9]:
epoch_type = 'evoked'
# epoch_type = 'pseudo-rs'


all_participants = ['S02_auditivo','S02_visual','S03_auditivo','S04_auditivo','S04_visual','S05_auditivo','S05_visual','S06_auditivo','S06_visual','S08_auditivo','S09_auditivo','S09_visual','S10_auditivo','S11_auditivo','S11_visual','S12_auditivo','S12_visual']


# path = '/media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/' #icm-linux
path = '/home/nicobruno/flor/Tesis/analisis/' 

# Compute markers for all subjects

In [10]:
target = 'epochs'

for i in all_participants:
    participant = i
    
    print('')
    print('#########################################')
    print(f'Computing markers for participant {participant}')
    print('#########################################')
    print('')
    

    #folder = path + participant +'/'

    
    df_markers = pd.DataFrame()
    df_markers['participant'] = participant
    
    #############################
    #### With ERP SUBTRACTED ####
    #############################
    #epochs_subtracted = mne.read_epochs(path + participant + '_' + epoch_type + '_' +  'ar_subtracted_epo.fif')
    epochs_subtracted = mne.read_epochs(path + participant + '_' + 'epochs' + '.fif')
#     epochs_subtracted = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'noica_ar_subtracted_epo.fif') 
    epochs_subtracted.info['description'] = 'biosemi/64' #necessary for wSMI 
    epochs_subtracted =  epochs_subtracted.pick_types(eeg = True) #EOGs break everything\
    
    ##################
    #### With ERP ####
    ##################
    #epochs_erp = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'ar_rereferenced_epo.fif')
#     epochs_erp = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'noica_ar_rereferenced_epo.fif')
    
    #epochs_erp.info['description'] = 'biosemi/64' #necessary for wSMI
    #epochs_erp =  epochs_erp.pick_types(eeg = True) #EOGs break everything
    
    #df_subtracted = pd.DataFrame.from_dict(all_markers(epochs_subtracted, 0, 0.6, target,epochs_erp= epochs_erp))
    df_subtracted = pd.DataFrame.from_dict(all_markers(epochs_subtracted, 0, 0.6, target))

    #df_subtracted = df_subtracted.assign(
    #events = epochs_subtracted.events[:,2],
    #label = lambda df: df.events.apply(lambda x: make_str_label(x)).str.split('/'), 
    #probe = lambda df: df.label.apply(lambda x: x[0]),
    #mind = lambda df: df.label.apply(lambda x: x[1]),
    #stimuli = lambda df: df.label.apply(lambda x: x[2]),
    #correct = lambda df: df.label.apply(lambda x: x[3]), 
    #prev_trial = lambda df: df.label.apply(lambda x: x[4]),
    #segment = lambda df: df.label.apply(lambda x: x[5]),
    #preproc = 'subtracted',
    #epoch_type = epoch_type
    #)
    
    df_markers = pd.concat([df_markers, df_subtracted], ignore_index=True)
        

    
#     df_erp = pd.DataFrame.from_dict(all_markers(epochs_erp, -0.2, 0.6, target))
    
#     df_erp = df_erp.assign(
#     events = epochs_erp.events[:,2],
#     label = lambda df: df.events.apply(lambda x: make_str_label(x)).str.split('/'), 
#     probe = lambda df: df.label.apply(lambda x: x[0]),
#     mind = lambda df: df.label.apply(lambda x: x[1]),
#     stimuli = lambda df: df.label.apply(lambda x: x[2]),
#     correct = lambda df: df.label.apply(lambda x: x[3]), 
#     prev_trial = lambda df: df.label.apply(lambda x: x[4]),
#     segment = lambda df: df.label.apply(lambda x: x[5]),
#     preproc = 'erp',
#     epoch_type = epoch_type
#     )
#     df_markers = df_markers.append(df_erp)
    
    
    folder = "/home/nicobruno/flor/Tesis/resultados/"
    df_markers.to_csv(folder+ participant + '_' + epoch_type + '_all_marker.csv')
#     df_markers.to_csv(folder+ participant + '_' + epoch_type + 'noica_all_marker.csv')



#########################################
Computing markers for participant S02_auditivo
#########################################

Reading /home/nicobruno/flor/Tesis/analisis/S02_auditivo_epochs.fif ...
    Found the data of interest:
        t =    -699.99 ...     499.99 ms
        0 CTF compensation matrices available
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Autodetected number of jobs 48
Effective window size : 8.192 (s)


/tmp/ipykernel_33038/4043092667.py:23: RuntimeWarning: This filename (/home/nicobruno/flor/Tesis/analisis/S02_auditivo_epochs.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_subtracted = mne.read_epochs(path + participant + '_' + 'epochs' + '.fif')
[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  11 out of  48 | elapsed:    4.5s remaining:   15.2s
[Parallel(n_jobs=48)]: Done  21 out of  48 | elapsed:    5.0s remaining:    6.5s
[Parallel(n_jobs=48)]: Done  31 out of  48 | elapsed:    5.2s remaining:    2.9s
[Parallel(n_jobs=48)]: Done  41 out of  48 | elapsed:    5.3s remaining:    0.9s


Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/be

[Parallel(n_jobs=48)]: Done  48 out of  48 | elapsed:    5.5s finished


Reduction order for nice/marker/PowerSpectralDensitySummary/summary_sef95: ['channels']
Running KolmogorovComplexity
K:: Using 1 threads to compute
K:: Buffer size: 264
Elapsed time 0.2414238452911377 sec
Reduction order for nice/marker/KolmogorovComplexity/default: ['channels']
Filtering  at 166.67 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Filtering  at 83.33 Hz


...

/tmp/ipykernel_33038/2076338759.py:232: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in an error because the following channel names are missing:
['AF3', 'AF4', 'F1', 'F2', 'FC1', 'FCz', 'FC2']
Either fix your included names or explicitly pass ordered=False.
  roi_cnv = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=cnv_chs))
/tmp/ipykernel_33038/2076338759.py:232: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  roi_cnv = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=cnv_chs))
/tmp/ipykernel_33038/2076338759.py:244: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in an error because t

Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/be

/home/nicobruno/nice/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: divide by zero encountered in log
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
/home/nicobruno/nice/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: invalid value encountered in multiply
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))


Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Filtering  at 41.67 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Filtering  at 20.83 Hz
Performing symbolic transformation
Reduction order for nice/marker/PermutationEntropy/default: ['channels']
Cannot autodetect number of jobs
Computing CSD
Fitted sphere radius:         99.3 mm
Origin head coordinates:      -1.1 14.1 38.4 mm
Origin device coordinates:    -1.1 14.1 38.4 mm
Filtering  at 166.67 Hz
Performing symbolic transformation
Running wsmi with python...
Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Cannot autodetect number of jobs
Computing CSD
Fitted sphere radius:         99.3 mm
Origin head coordinates:      -1.1 14.1 38.4 mm
Origin device coordinates:    -1.1 14.1 38.4 mm
Filtering  at 83.33 Hz
Performing symbolic transformation
Running wsmi with python...
R

/tmp/ipykernel_33038/2076338759.py:232: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in an error because the following channel names are missing:
['AF3', 'AF4', 'F1', 'F2', 'FC1', 'FCz', 'FC2']
Either fix your included names or explicitly pass ordered=False.
  roi_cnv = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=cnv_chs))
/tmp/ipykernel_33038/2076338759.py:232: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  roi_cnv = np.array(mne.pick_channels(epochs_erp.info['ch_names'], include=cnv_chs))
/tmp/ipykernel_33038/2076338759.py:244: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in an error because t

In [18]:
path = '/home/nicobruno/flor/Tesis/resultados/'
sujeto = path + 'S04_auditivo_evoked_all_marker.csv'

from numpy import genfromtxt
lista=2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,5,26,27,28,29,30
my_data= genfromtxt(sujeto, skip_header=1,usecols=(lista),delimiter=',')

In [19]:
my_data.shape

(400, 29)

In [20]:
my_data

array([[ 1.65041908e-02,  1.67404529e-02,  3.18854020e-02, ...,
         2.81283779e+00, -2.66514726e+00, -3.69001563e+00],
       [ 1.68556695e-02,  2.24617361e-02,  3.01364099e-02, ...,
         3.64753505e+00,  3.37330209e+00, -6.24756475e+00],
       [ 1.59081363e-02,  1.53626118e-02,  2.99756070e-02, ...,
        -8.83310363e+00, -4.70835767e+00, -4.67232134e+00],
       ...,
       [ 1.48417495e-02,  1.65109671e-02,  2.69246229e-02, ...,
        -1.68705716e+00,  8.06029059e-01, -4.56440827e+00],
       [ 1.40382502e-02,  8.25744392e-03,  1.14357522e-02, ...,
         2.86398952e+00, -1.20444014e-01, -8.61733070e+00],
       [ 1.48356485e-02,  1.75295022e-02,  2.95925389e-02, ...,
         9.54927344e+00,  7.19259006e+00, -4.49233168e+00]])

In [22]:
labels= genfromtxt('/home/nicobruno/flor/Tesis/vectores/S_04_auditivo.csv', skip_header=1,delimiter=',')

In [23]:
labels

array([1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 0.

In [41]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from random import shuffle

auc = []
auc_sh = []

n_iter = 10

features = np.zeros((n_iter,29))
features_sh = np.zeros((n_iter,29))

for i in np.arange(n_iter):
    print(i)
    temp = []
    temp_sh = []

    ftemp = np.zeros((5,29))
    ftemp_sh = np.zeros((5,29))

    kf = KFold(n_splits=5)
    n=-1
    for train, test in kf.split(my_data):
        n=n+1
        X_train, X_test, y_train, y_test = my_data[train], my_data[test], labels[train], labels[test]
        clf = RandomForestClassifier(n_estimators=100)
        clf.fit(X_train,y_train)
        probas = clf.predict_proba(X_test)
        temp.append(roc_auc_score(y_test,probas[:,0]))
        ftemp[n,:] = clf.feature_importances_

        y_train_sh = y_train.copy()
        shuffle(y_train_sh)
        clf.fit(X_train,y_train_sh)
        probas = clf.predict_proba(X_test)
        ftemp_sh[n,:] = clf.feature_importances_
        temp_sh.append(roc_auc_score(y_test,probas[:,0]))

    auc.append(np.mean(np.array(temp)))
    auc_sh.append(np.mean(np.array(temp_sh)))
    features[i,:] = np.mean(ftemp,0)
    features_sh[i,:] = np.mean(ftemp_sh,0)






0
1
2
3
4
5
6
7
8
9


In [54]:
nf = 4

np.sum(features_sh[:,nf]>features[:,nf])/10

np.float64(0.1)